# 09/10/20
Post-vacation contact with supervisor - added to Slack and given paper on Bayesian Machine learning in metamaterial design to read (https://onlinelibrary-wiley-com.ezp.lib.cam.ac.uk/doi/full/10.1002/adma.201904845#adma201904845-fig-0002).

# 11/10/20
Testing GPyOpt functionality with their manual examples and functions from https://en.wikipedia.org/wiki/Test_functions_for_optimization. 1D and 2D straightforward with visuals. Tried with higher dimensional problems (n>=10) - sphere function was fine, but less optimal when the function is rough and needs more iterations (local minima).

# 12/10/20
Formalise testing - see gpyopt_test.py
Added timing - 15 iterations for 2D Booth function and 10D sphere function both took around 5s to run, 40 iterations for 10D Rastrigin function took 13s with f(x) error of 0.8. Not bad!
Made github repo for project.

# 13/10/20
Call with supervisor to outline next steps:
* Further testing with GPyOpt
    * evaluation time, no. of evaluations to optimum
    * compare different acquisition functions
    * test with non-linear functions (introduce discontinuities)
    * sparse Gaussian processes to lower compute time
* Consider multi-fidelity Gaussian processes - data from multiple sources
* Look into deep learning methods for optimisation - fast.ai

# 25/10/20
Acquisition function $$u(x)$$ guides finding new samples in objective function. Balance exploration (samples from unexplored/high uncertainty areas) and exploitation (samples around current optimum, drill down to true (local) optimum). Formally, take sample where $$\mathbf{x}=\underset{\mathbf{x}}{\arg \max} u(x|\mathcal{D}_{1:t-1})$$ where $$\mathcal{D}_{1:t-1}$$ is data up to this point. Most common acquisition functions are Maximum Probability of Improvement (MPI), Expected Improvement (EI) and Upper Confidence Bound (UCB).

Testing of GPyOpt - evaluation time and accuracy when varying noise, maximum iterations, acquisition function, dimension, function type (convex?). Results given in gpyopt_test_results.xlsx

Biggest factors in accuracy loss, ceteris paribus:
* roughness of function, gets stuck in local minima for a long time
* number of dimensions/parameters, high dimensional inputs take longer to calculate/train to good accuracy

Still to do: 
* testing with non-linear functions
* investigate sparse GP (can test very easily but what is it doing differently behind the scenes?)
* set up multi-fidelity GP

Can parallelise optimisation - biggest bottleneck in evaluating objective function (according to GPyOpt documention). This needs further investigation to find where it can actually help, especially in reference to batches.

Spoke to Tianyi (PhD student) about her finite element model code in C++. Will have this resource to output results but will need integrating into Python. Consider adapting code/look into existing Python libraries (e.g. SfePy).

Thoughts on presentation outline: 
* introduce and walk through Gaussian Processes for 1D/2D
* give example where optimization might be useful (minimise weight for a component while keeping strength above certain threshold)
* connect GP with structures
* pros and cons

Report can probably also follow this general outline, with more specific examples and experiments to show optimisation in practice for simple and complex structures. Aim to have end result of a useable modular code base that can be used to perform GP optimisation on any defined structure problem.

# 26/10/20
Action points coming out of call with supervisor:
* use relative errors
* does optimisation slow with number of iterations? (should do from covariance matrix getting larger and harder to invert)
* test sparse GP as way to combat costly computations
* look into python FEM libraries
* start thinking about structural problems to centre final experimentation around
* test multi-fidelity GP

Make sure to keep bank of images/graphs to put in report, systematically go through interest points and build up story.

# 01/11/20
Relative error wrt true **X** not possible when true **X** is 0. Similar for f(**X**). (For real structural optimisation problems this is unlikely to be a problem.) 2 options:
* Measure relative error percentage wrt benchmark error - GP model always imperfect, what can we accept as "expected error"? 
* Add constant component to true function to work around minimum f(**X**)=0 problem.

First option encodes error tolerance, which is useful for real problems. Optimising certain variables accurately has smaller influence on the final result than other variables. To measure relative error fairly, this benchmark error should change with complexity of problem. We can also choose benchmark error as the average error.

Second option is an easy work-around which does not require us to choose a benchmark error. However the added constant component would be arbitrary and not add information, just a way to turn errors into percentage errors.

Performed additional testing with sphere and rastrigin functions at 10-D. Fixing everything else, I varied the number of iterations and recorded the time taken per iteration. I repeated this for 5 rounds and plotted the averages. From the graph in Sheet 6 of gpyopt_test_results.xlsx, can see that it is obvious for the sphere function that optimisation slows with the number of iterations. However, this is not as clear for the rastrigin function - there is still a vague trend showing the optimisation shows, but the graph is much more level. Not sure why this is? **something to look into

Sparse GP supposed to speed up process of inverting covariance matrix but not seeing this, in fact seeing a significant slow down in time taken to run a fixed number of iterations. Why is this?

SfePy can be used but need to figure out how to generate new meshes - ask supervisor about this. Everything specified in problem description file and solved with one command (easy to use but hard to debug).

# 03/11/20
Sparse GP issue was coding bug, now seeing speed up from using sparseGP model. Seems to be useful for fitting to lots of data points but not for doing optimization where it uses acquisition function to find new points. Why? 2866s to optimise with 10,000 data points (2D) with sparseGP, several hours and still not complete for GP.


# 04/11/20
Action points out of meeting with supervisor:
* look at GPy to check what sparse GP model is doing
* look into PyNite for FEM (https://github.com/JWock82/PyNite)
* test with constrained functions (constrain volume of materials etc)
* do first draft of presentation for next week

# 12/11/20
First draft of presentation completed. Sent to supervisor for content check, aim to run through at next meeting before presentation day (26/11/20).
sparseGP model needs inducing samples that are drawn from X and Y inputs to BayesianOptimisation function, so cannot be run just using acquisition function. Idea is to summarise large amount of data using a small numer of pseudo-data (e.g. highlight peaks and valleys). Uses Variational Free Energy model - approximation of posterior distribution, minimising KL divergence between the 2 functions. Need to dig into this more.
(References for later: 
* https://towardsdatascience.com/sparse-and-variational-gaussian-process-what-to-do-when-data-is-large-2d3959f430e7
* http://proceedings.mlr.press/v5/titsias09a/titsias09a.pdf
* https://www.youtube.com/watch?v=sQmsQq_Jfi8
* http://gpss.cc/gpss17/slides/gp-approx-new.pdf
)


# 17/11/20
Action points:
* edits on presentation
* connect FEM and GP
* multi-fidelity GP - look at papers, extend library?
    * low fidelity by treating structural parts independently
    * high fidelity - only take a few points of super complex model
    
Major bottleneck in structural optimisation is actually is calculating points - running FEM models takes a long time. So sparseGP might not actually be best optimisation route, want to consider multi-fidelity GP instead. Takes lots of samples of a function that is cheap to evaluate and a few ssamples of a related (expensive) function, combine data sets to get better estimate of expensive function.

# 21/11/20
References for multi-fidelity GP:
* https://arxiv.org/pdf/1604.07484.pdf
* https://arxiv.org/pdf/1603.06288.pdf
* https://icerm.brown.edu/materials/Abstracts/tw-17-4/Gaussian_processes-_A_hands-on_tutorial_]_Paris_Perdikaris,_Massachusetts_Institute_of_Technology.pdf
* https://github.com/paraklas/GPTutorial/tree/master/code

How to get low fidelity data points for structural problems?
https://caendkoelsch.wordpress.com/2017/05/20/ways-to-reduce-the-computational-time-in-a-fem-simulation/#:~:text=The%20first%20and%20most%20common,a%20step%20in%20the%20simulation.
* FEM with larger (and fewer) finite elements - less computation time, more error
* adding rigid body constraints may apply for some problems
* adding symmetry



# 22/11/20
Set up a function using PyNite to output the max deflection and total weight of an example structure - what is the objective function? Want to minimise wrt both jointly somehow - add two terms with a given variable to set relative cost. (Can also try to figure out how to do multi output GP optimisation. This is related to multi-fidelity GP so worth thinking about anyway.)

# 23/11/20
Multifidelity code from GPTutorial - REMEMBER TO REFERENCE. Need to check maths and try to expand to multi-dim since throwing errors right now. If this fails try MatLab code from https://github.com/maziarraissi/TutorialGP.

Action points from meeting with Fehmi:
* Edits in presentation for Thrusday
* Write out VTK Polydata to vtu file to read in Paraview - figure out how to do this (writepolydata?)
* Do multifidelity in >1D

# 29/11/20
Got multifidelity GP working in >1D. Tested with Booth and Sphere functions and obtained fairly low L2 errors. Visualised prediction vs actual functions in 2D and saw good fit.

# 01/12/20
https://arxiv.org/pdf/1912.00392.pdf gives experimental plan using multi-fidelity GP
Experiment plan:
* find a structure of interest and create model in PyNite
* create simplified model for low-fidelity data
* experiment design to find where to take initial points - obtain initial high fidelity data
* similarly obtain initial low fidelity data
* use acquisition function on low-fidelity to find next point
* use acquisition function on high-fidelity to find next point
* repeat until satisfied
* find min y_pred+y_var using multi-fidelity GP

http://auai.org/uai2019/proceedings/papers/284.pdf for a multi-fidelity friendly acquisition function

# 02/12/20
Action Points:
* figure out how to make bayesian multi-fidelity - see 01/12/20
* find structures problem to experiment with - preferably high texture/complex so easy way to make low-fidelity mdoel

# 16/01/21
* https://arxiv.org/pdf/1012.2599.pdf for detailed description of acqusition functions
* http://proceedings.mlr.press/v89/song19b/song19b.pdf or https://arxiv.org/pdf/1603.06288.pdf for multifidelity framework
* https://www-sciencedirect-com.ezp.lib.cam.ac.uk/science/article/pii/S026412752030085X for an possible final test structure - experiment laid out but uses different optimisation method, nice for compare and contrast

# 18/01/21
Completed Technical Milestone Report - sent to supervisor for read-through and feedback to hand-in on 21/01/21.

# 24/01/21
Starting to write truss model for final test structure. Some questions - what do I do with members that overlap? How to do varying number of non-basic nodes for optimisation? (maybe do a number of nodes at a time and then create pareto frontier) How to make this actual 3D element model???

# 01/02/21

Finished writing code for truss model, haven't dealt with overlapping members really since the truss model has infinitesimally thin members so ignoring that for now. Inputs are just number of units, list of non-basic node classes (includes name and relative position to origin of a unit) and list of member property classes. Will change that to calculate the properties based on input dimensions later. For the optimisation, I want to optimise across number of non-basic nodes. However, the input list for member properties has to be constant. Potential solution: set maximum number of non-basic nodes, make the list the appropriate length for that and set some to zero. Will need to test this, might be easier just to do local optimisations and plot the results to estimate global optimum.

For the optimisation, should do 2 types and compare - constrain weight and minimise deflection or a weighted sum of the 2, varying weights to get Pareto frontier.

Downloaded ABAQUS on Windows laptop and can maybe use that to do the 3D element model? Can be a check on FEA results from PyNite and obtain some "high-fidelity" results by hand.

# 04/02/21
Effective EIs calculated using different tip loads are the same in PyNite (as expected). EI calculated as $$\frac{F_{tip}L^3}{3\Delta_{max}}$$